# Prompt Optimizer

In this notebook we are building a prompt optimizer based upon the research whitepaper:  Principled Instructions Are All You Need for Questioning LLaMA-1/2, GPT-3.5/4  download from here https://arxiv.org/pdf/2312.16171.pdf


In [ ]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
%pip install --upgrade openai

In [2]:
# import os module & the OpenAI Python library for calling the OpenAI API
import os
from openai import AzureOpenAI
import json
from dotenv import load_dotenv
load_dotenv()

True

# Setup Parameters

In [3]:
# Expecting an .env file with the following content:
# OPENAI_API_KEY=your_openai_api_key_here
# AZURE_OPENAI_ENDPOINT ="https://your_open_ai_resource_name.openai.azure.com/"



In [4]:
# YOU CAN USE https://huggingface.co/spaces/microsoft/llmlingua-2 TO COMPRESS YOUR PROMPTS 
# IN THIS EXAMPLE WE ARE USING THE FOLLOWING PROMPT and we processed a compressed version and the prompt got reduced by 25% 

In [5]:

# System prompt for reviewing user prompts, adapted for the conversational format
#722 tokens:
reviwer_system_prompt = """
##Objective##:
To review user prompts critically, ensuring they align with the 26 key principles for effective interaction with Large Language Models (LLMs), and to provide constructive feedback for improvement.

##Instructions for the Assistant##
#Directness and Conciseness: Confirm the user prompt is straightforward, avoiding unnecessary politeness or filler words.
#Intended Audience: Ensure the user prompt clearly identifies who the audience is, tailoring the complexity and style accordingly.
#Simplifying Complex Tasks: Check if complex tasks are broken down into simpler, sequential prompts.
#Affirmative Directives: Assess the use of clear, action-oriented language, avoiding negative constructions.
#Clarity in Requests: Verify if the prompt asks for explanations in simple terms, suitable for laypersons or specific age groups.
#Incentive Mention: Note if the prompt includes unnecessary incentives, which are not applicable here.
#Example-Driven: Ensure the prompt encourages or includes specific examples or few-shot learning approaches.
#Structured Formatting: Check for clear, structured formatting, including instructional headers and separation of sections.
#Directive Phrasing: Confirm the use of explicit directives like “Your task is” to guide the LLM's response.
#Consequence Awareness: Look for language indicating awareness of the importance of accuracy and the potential consequences of misinformation.
#Natural, Human-Like Responses: Assess whether the prompt requests responses in a natural, conversational manner.
#Step-by-Step Guidance: Ensure the prompt encourages thinking or explaining processes step by step.
#Bias Avoidance: Check for explicit instructions to avoid bias and rely on stereotypes.
#Detail Elicitation: Confirm if the prompt is open-ended, allowing the LLM to ask follow-up questions for clarification.
#Learning and Testing: Verify if the prompt encourages an educational approach, including learning checks or tests.
#Role Assignment: Assess whether the prompt assigns a specific role or perspective to the LLM.
#Use of Delimiters: Check for the use of delimiters for clarity in separating sections or ideas.
#Keyword Repetition: Look for intentional repetition of key words or phrases for emphasis.
#Combining CoT with Few-Shot: Ensure the prompt integrates a Chain-of-Thought approach with example-driven or few-shot learning for complex tasks.
#Output Primers: Confirm if the prompt concludes with an output primer, setting up the expected format or beginning of the response.
#Detailed Writing Requests: Assess if the prompt asks for detailed, comprehensive coverage on a topic.
#Style Preservation: Ensure the prompt asks for stylistic consistency when revising or responding to user inputs.
#Complex Coding Requests: Note if the prompt properly guides the creation of multi-file coding solutions or complex coding tasks.
#Content Initiation: Verify if the prompt effectively sets the stage or beginning for creative or continuation tasks.
#Clear Requirements: Ensure the prompt clearly outlines all requirements, including format, style, and specific content expectations.
#Style Mimicry: Check if the prompt asks for mimicry of a provided sample's style or language for text generation tasks.

##Feedback and Recommendations##

#Provide specific, actionable feedback for each principle, noting where the user prompt excels and where it falls short.
#Offer suggestions for improvement, ensuring the user understands how to align their prompt better with these principles.
#YOU MUST ALWAYS GENERATE AN IMPROVED VERSION OF THE PROMPT AND SUGGEST IT TO THE USER


**Self-evaluation:**
- Clarity and Brevity: The response is clear and provides a concise summary of what should be included when summarizing historical financial documents. It avoids unnecessary jargon and focuses on the most relevant financial metrics. (9/10)
- Accuracy: The answer is internally consistent and accurately reflects the type of information typically found in historical financial summaries.

##Final Evaluation##
ADD KEYWORDS SUCH AS COMPANY NAME, TARGET USERS, TOPIC IN BETWEEN {} TO MAKE THE PROMPT MORE SPECIFIC WHENEVER YOU HAVE TO TO FOLLOW THE 26 GUIDLINES
Give an overall evaluation of the prompt's effectiveness and adherence to the 26 principles.
Encourage the user to refine their prompting skills based on the feedback.

"""

#518 tokens:
reviwer_system_prompt_compressed = """
Objective 
 review user prompts, align with 26 principles for interaction with Large Language Models ( LLMs, provide constructive feedback for improvement. 
 Instructions for Assistant 
 Directness and Conciseness Confirm user prompt straightforward, avoiding unnecessary politeness filler words. 
 Intended Audience Ensure user prompt identifies audience, tailoring complexity style. 
 Simplifying Complex Tasks Check complex tasks broken into simpler, sequential prompts. 
 Affirmative Directives Assess clear, action - oriented language, avoiding negative constructions. 
 Clarity in Requests prompt asks for explanations in simple terms, suitable for laypersons age groups. 
 Incentive Mention prompt includes unnecessary incentives, not. 
 Example - Driven prompt encourages specific examples learning approaches. 
 Structured Formatting, including instructional headers separation of sections. 
 Directive Phrasing Confirm use explicit directives guide LLM response. 
 Consequence Awareness language awareness of importance of accuracy potential consequences of misinformation. 
 Natural, Human - Like Responses Assess prompt requests responses natural, conversational manner. 
 Step - by - Step Guidance encourages thinking explaining processes step by. 
 Bias Avoidance Check for explicit instructions to avoid bias rely on stereotypes. 
 Detail Elicitation prompt open - ended, LLM ask follow - up questions for clarification. 
 Learning and Testing encourages educational approach, including learning checks tests. 
Role Assignment prompt assigns specific role perspective to LLM. 
 Use of Delimiters for clarity in separating sections ideas. 
 Keyword Repetition intentional repetition of key words phrases for emphasis. 
 Combining CoT with Few - Shot integrates Chain - of - Thought approach with example - driven few - shot learning for complex tasks. 
 Output Primers prompt concludes with output primer, setting expected format or beginning response. 
 Detailed Writing Requests detailed, comprehensive coverage topic. 
 Style Preservation stylistic consistency revising responding to user inputs. 
 Complex Coding Requests guides creation of multi - file coding solutions complex coding tasks. 
 Content Initiation prompt sets stage or beginning for creative continuation tasks. 
 Clear Requirements outlines requirements, including format, style, specific content expectations. 
 Style Mimicry mimicry of provided sample ' s style or language for text generation tasks. 
 Feedback and Recommendations 
 Provide specific, actionable feedback for each principle, user prompt excels falls short. 
 Offer suggestions for improvement, user understands align prompt better with principles. 
 MUST ALWAYS GENERATE IMPROVED VERSION OF PROMPT SUGGEST IT TO USER 
 YOU MUST ALWAYS INCLUDE THIS SECTION AT THE END:
 
**Self-evaluation:**
- Clarity and Brevity: The response is clear and provides a concise summary of what should be included when summarizing historical financial documents. It avoids unnecessary jargon and focuses on the most relevant financial metrics. (9/10)
- Accuracy: The answer is internally consistent and accurately reflects the type of information typically found in historical financial summaries.
 Final Evaluation 
 ADD KEYWORDS SUCH AS COMPANY NAME, TARGET USERS, TOPIC IN BETWEEN MAKE PROMPT MORE SPECIFIC WHENEVER HAVE FOLLOW 26 GUIDLINES 
overall evaluation prompt ' effectiveness adherence to 26 principles. 
 Encourage user refine prompting skills based feedback.
 

"""

prompt_to_review = """
Develop an executive summary that encapsulates the potential investment in a point-of-care testing company within the in-vitro diagnostics market. The summary should:

Present an overview of the company's business model, product offerings, market reach, and financial highlights without specifying exact sales figures or margins.
Give an account of the company's historical background, current ownership, and ongoing bid process, mentioning interest from other equity firms without naming them.
Formulate an investment thesis that emphasizes the company's market position, growth strategy, and business model strengths, such as recurring revenue streams and proprietary technology.
List key investment considerations and due diligence items that an investor should evaluate, focusing on market share, product portfolio, and technological innovation.
Conclude with a preliminary valuation outlook and recommendations for proceeding with an investment procedure, without including specific multiples or revenue projections."
One-shot Example for LLM Output:

"The executive summary should highlight an investment opportunity in a dynamic IVD company with a strong presence in point-of-care testing. The company, leveraging a recurring revenue model, has established significant market access in Europe and plans for global expansion. Originally part of a larger pharmaceutical entity, it is now in the midst of a strategic acquisition process. The investment thesis should underscore the firm's solid market standing, the potential for growth via technological advancements, and its strategic plans for market expansion. Considerations for investment should cover market analysis, risk assessment, and evaluation of proprietary technologies. The summary concludes with a recommendation for investment proceedings based on a positive valuation outlook, considering the company's robust business model and future potential."
"""
messages =[
    {
        "role": "system",
        "content": reviwer_system_prompt_compressed
    },
    {
        "role": "user",
        "content": prompt_to_review
    }
]
   



In [6]:

try:
    
    client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-15-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

    deployment_name='gpt-4' #This will correspond to the custom name you chose for your deployment when you deployed a model. Use a gpt-35-turbo-instruct deployment. 
        
    # Send a completion call to generate an answer
    completion = client.chat.completions.create(
        messages=messages,
        model=deployment_name,
        max_tokens=3000
    )

    print(f"{completion.choices[0].message.role}: {completion.choices[0].message.content}")
except Exception as e:
    # Handles all other exceptions
    print(f"An exception has occurred: {e}")


assistant: Feedback and Recommendations:
1. Directness and Conciseness: The user prompt is direct and concise in its request for a summary. No improvements needed.
2. Intended Audience: The prompt is tailored for an executive audience. It's advisable to use industry-specific language appropriate for professionals in finance and investment. 
3. Simplifying Complex Tasks: The task is relatively complex but not broken into simpler steps. It could benefit from an instructional phrase like "Begin with providing an overview of..." for each requested section.
4. Affirmative Directives: The use of imperative sentences (“Present”, “Give”, “Formulate”, “List”, “Conclude”) is clear and affirmative. No improvements needed.
5. Clarity in Requests: Clarity can be improved by specifying a layperson or a more detailed age group if the audience is beyond industry professionals.
6. Incentive Mention: There is no mention of incentives in the prompt; this is appropriate since incentives are not necessary.

#Remember that you can process this system prompt with LLMLingua2 and reduce the size.
#For instance this was the ouptut of one of the executions:

"Your task is to create an executive summary focusing on an investment in {Company X}, a point-of-care testing organization within the in-vitro diagnostics industry. Please include the following sections:

1. **Business Overview**: Summarize the company's model, products, market reach, and financial highlights (excluding specific figures).
2. **Historical Context**: Describe the company's origins, current ownership, and hint at the acquisition interest.
3. **Investment Thesis**: Craft a compelling argument highlighting market position, growth, and business model strengths, such as recurring income and unique technology.
4. **Investment Considerations**: List the important factors an investor should review, including market share and innovation.
5. **Valuation and Recommendations**: Offer a general valuation perspective and suggest next steps for potential investors. Avoid exact financial forecasts or multiples.

Provide clear headings for each section and write in a professional tone suitable for an investor audience. Avoid naming specific equity firms or revealing confidential financial details. Instead, focus on presenting qualitative insights and strategic value propositions that portray the company's investment potential in a balanced and informative manner."


After processing this prompt using LINGUA2 https://huggingface.co/spaces/microsoft/llmlingua-2
Original prompt token lenght:  224 
Compressed token lenght: 170


OUtput of the compression:

task create executive summary investment in Company X }, point - of - care testing organization in - vitro diagnostics industry. include sections : 
 1. Business Overview Summarize company ' s model, products, market reach, financial highlights excluding specific figures. 
 2. Historical Context Describe company ' s origins, current ownership, hint acquisition interest. 
 3. Investment Thesis Craft compelling argument highlighting market position, growth, business model strengths, recurring income unique technology. 
 4. Investment Considerations List important factors investor review, market share innovation. 
 5. Valuation and Recommendations Offer general valuation perspective suggest next steps for potential investors. Avoid financial forecasts. 
 Provide clear headings for each section write professional tone for investor audience. Avoid naming specific equity firms confidential financial details., focus on presenting qualitative insights strategic value propositions company investment potential balanced informative.
